<a href="https://colab.research.google.com/github/mahdibaghbanzadeh/NLP/blob/main/notebooks/03_text_classification_using_sentence_embedding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources:

*   https://www.sbert.net/index.html
*   https://www.sbert.net/docs/pretrained_models.html




In [1]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
np.random.seed(2022)

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/MyDrive/data/toxic_comments/train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [14]:
df_sam = data.sample(frac=0.01)
df_sam.shape

(1596, 8)

In [8]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'me', 'wherein', '’ve', 'twenty', 'fifteen', 'we', 'forty', 'thence', 'even', 'call', 'next', 'regarding', 'cannot', 'any', 'still', 'make', 'since', 'although', 'the', 'former', 'often', 'via', 'both', '’ll', 'very', 'now', '’s', 'many', 'moreover', 'while', 'mostly', 'below', 'seem', 'what', 'if', 'fifty', 'few', 'they', '’re', 'all', 'as', 'top', 'your', 'not', 'been', 'really', 'than', 'yours', 'n‘t', 'of', 'someone', 'used', 'had', 'between', 'each', 'latter', 'no', 'under', 'you', 'that', 'did', 'towards', 'nine', 'already', 'side', 'being', 'seemed', 'something', 'with', 'amount', '’m', 'hereby', 'none', 'whole', 'eleven', 'hers', 'besides', 'amongst', 'these', 'whom', 'could', 'thereupon', 'in', 'unless', 'within', 'at', 'our', 'less', 'almost', 'ten', 'across', 'alone', '‘ve', 'up', 'mine', 'enough', 'where', 'put', 'twelve', 'though', 'ca', 'so', 'various', 'be', 'yourself', 'there', 'over', 'about', 'sometimes', 'became', 'thereafter', 'see', 'n’t', 'last', 'its', '‘s', 'hi

In [9]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [10]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)


    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    sentence = " ".join(mytokens)
    # return preprocessed list of tokens
    return sentence

In [11]:
list_ = ["happy" , "apple"]
"-".join(list_)

'happy-apple'

In [12]:
spacy_tokenizer("I am Eating")

'eat'

In [15]:
df_sam['tokenize'] = df_sam['comment_text'].apply(spacy_tokenizer)

In [16]:
df_sam.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokenize
67713,b528688dc45b4f61,"""== November 2007 ==\n\nA tag has been placed ...",0,0,0,0,0,0,november 2007 tag place rags bowen request spe...
741,02004db8cf8f91f0,"As a rollbacker, that is particularly problema...",0,0,0,0,0,0,rollbacker particularly problematic aware edit...
110911,5161525711ed58fc,What's going on Haminoon is you are quite clea...,0,0,0,0,0,0,haminoon clearly campaign manufacture death ir...
4912,0cff83251f6a4cab,You are my bitch! \n\nI own your ass.,1,1,1,0,1,0,bitch ass
23986,3f5c349b73f4211c,That´s your point of view. I understand that y...,0,0,0,0,0,0,that´s point view understand got ban behaviour...


In [17]:
df_sam['embeddings'] = df_sam['tokenize'].apply(model.encode)

In [18]:
df_sam.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokenize,embeddings
67713,b528688dc45b4f61,"""== November 2007 ==\n\nA tag has been placed ...",0,0,0,0,0,0,november 2007 tag place rags bowen request spe...,"[-0.0065389243, -0.012369587, 0.02263211, 0.00..."
741,02004db8cf8f91f0,"As a rollbacker, that is particularly problema...",0,0,0,0,0,0,rollbacker particularly problematic aware edit...,"[-0.022191249, 0.027706124, -0.03913033, 0.060..."
110911,5161525711ed58fc,What's going on Haminoon is you are quite clea...,0,0,0,0,0,0,haminoon clearly campaign manufacture death ir...,"[-0.019821808, -0.01092645, -0.060690887, -0.0..."
4912,0cff83251f6a4cab,You are my bitch! \n\nI own your ass.,1,1,1,0,1,0,bitch ass,"[-0.026483461, -0.038179684, -0.001860707, 0.0..."
23986,3f5c349b73f4211c,That´s your point of view. I understand that y...,0,0,0,0,0,0,that´s point view understand got ban behaviour...,"[-0.071037926, -0.024728702, 0.026169302, -0.0..."


In [19]:
X = df_sam['embeddings'].to_list()
y = df_sam['toxic'].to_list()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [21]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [22]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.93125
Logistic Regression Precision: 1.0
Logistic Regression Recall: 0.26666666666666666
